# Semantic Segmentation using SegNet 
## Introduction
This notebook is a summary of a personal project demonstrating how convolutional neural networks (CNNs) can be used for an image segmentation task. Specifically, we describe semantic labelling of specific targets within a dataset. Semantic segmentation is a common segmentation technique where different objects are distinguised from each other based on class type only, though each instance within the class is not delineated. 

<br>
<img src="typesofseg.png" width=1000 height=1000 />

##  Dataset 
I've been particularly interested in how supervised learning can be used in digital pathology and other medical/research image modailties. For this project, I used a **darkfield microscopy dataset** populated with labelled red blood cells (RBC's) and Spirochaeta bacteria. My goal was to construct and test a CNN architecture from the groud up which could label RBC's from bacteria within reasonable accuracy. For me, automating labelling of bacterium species seemed like a great starting point for exploring the capabilities of supervised learning in a diagnostic capacity. 

### Images and mask labels

A few examples from the dataset of of colorized cells and bacterium species (left) and corresponding labelled masks which would be used for training (right) 
<br>
<img src="1.png" width=500 height=500 />
<img src="2.png" width=500 height=500 />
<img src="3.png" width=500 height=500 />

### Overview
<img src="overview.png" width=800 height=800 />

## SegNet 
SegNet is an CNN configuration originally developed at the University of Cambridge for semantic segmentation. At the time of its publication, SegNet introduced a rather novel **encoding-decoding architecture** compared to its contemporaries, which used fully connected layers (and consequently many more parameters to train.) As for why I chose this architecture, it came down to my personal interest with its implementation. 

### Architecture
The encoding - decoding stragety aims to reduce the number of learnable parameters by sucessively reducing spatial resolution in the encoder stage, followed by restoring this resolution for the final label projection. We trade spatial resolution for feature map depth at each encoder stage to encourage the network to infer more complex abstractions (ie. patterns in the image data.) The decoder network inverts this process by restoring resolution and collapsing feature map depth to project a final prediction. 

The complete architecture is illustrated below from the original SegNet paper: 
<img src="segnet_architecture.png" width=900 height=900 />

- **Convolutional layer**: 


- **Batch normalization**: 


- **ReLU**: 


- **MaxPooling**: 


- **Max Unpooling**: 


- **Softmax**:  